# License

In [1]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg -y
    #!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
    #!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz
#installDeps()
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from audioInput import getLaughTracks, getNoise

def trainAndSaveAndPredict(test_data, model, number_of_classes = 2, number_of_samples = 1, epochs = 5, getData = getLaughTracks, use_cache = True, log = True):
    model_name = '%s_%s' % (model, number_of_samples)
    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, use_cache = use_cache, log = log)
    preds = train(curriedGetSamples, number_of_classes, model_name = model_name, epochs = epochs)
    
    return predict('%s_%s-%s' % (model_name, epochs, epochs), number_of_classes, test_data)

def printResults(preds, expected = None): 
    with tf.Graph().as_default(), tf.Session() as sess:
        print(preds)
        print(sess.run(tf.argmax(input=preds, axis=1))) 
        print('expected results', expected)

def trainForNoise(number_of_samples=5, epochs=5):
    use_cache = False
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, 'noise', number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=5, epochs=5):  
    use_cache = False
    #print('training on laughter and not laughter')
    (features, labels) = getLaughTracks(shuf=False, number_of_samples = 2, use_cache = use_cache, log=False)
    preds = trainAndSaveAndPredict(features, 'audio', number_of_classes = 2, number_of_samples = number_of_samples, epochs = epochs, getData = getLaughTracks, use_cache = use_cache, log = False)
    printResults(preds, [0, 0, 1, 1])


In [2]:
trainForLaughter(number_of_samples=1, epochs=1)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.72361
loading ./model/audio_1_1-1/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/audio_1_1-1/model
[[0.58894306 0.63782966]
 [0.60774016 0.63660455]
 [0.56984234 0.6090671 ]
 [0.5945718  0.6311385 ]]
[1 1 1 1]
expected results [0, 0, 1, 1]


In [3]:
#for i in range (0, 30):
#    number_of_samples = 1006 + (i * 1)
#    trainForLaughter(number_of_samples=number_of_samples, epochs=1)

In [ ]:
trainForLaughter(number_of_samples=500, epochs=100)

INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.690164
Step 2: loss 0.665999
Step 3: loss 0.634396
Step 4: loss 0.612395
Step 5: loss 0.532513
Step 6: loss 0.570745
Step 7: loss 0.567581
Step 8: loss 0.537245
Step 9: loss 0.458661
Step 10: loss 0.467219
Step 11: loss 0.603286
Step 12: loss 0.433018
Step 13: loss 0.388321
Step 14: loss 0.412329
Step 15: loss 0.447394
Step 16: loss 0.390374
Step 17: loss 0.337031
Step 18: loss 0.381813
Step 19: loss 0.298182
Step 20: loss 0.337973
Step 21: loss 0.275488
Step 22: loss 0.278766
Step 23: loss 0.266147
Step 24: loss 0.328608
Step 25: loss 0.186947
Step 26: loss 0.211109
Step 27: loss 0.263777
Step 28: loss 0.235831
Step 29: loss 0.198745
Step 30: loss 0.329258
Step 31: loss 0.303927
Step 32: loss 0.231897
Step 33: loss 0.202584
Step 34: loss 0.34923
Step 35: loss 0.100973
Step 36: loss 0.0894409
Step 37: loss 0.328973
Step 38: loss 0.131816
Step 39: loss 0.212398
Step 40: loss 0.300236
Step 41: loss 0.184675
Step